In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('/home/shashank/Desktop/IIT_D ML/prerequisite_assignment2/data3.csv',header=None)
data.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.968476,0.698022,0.275840,0.269753,0.402714,-0.704977,-0.927612,0.401272,-1.085193,0.427316,2.110024,2.313180,-0.740717,-0.298564,1.287575,-0.839947,0.023128,-1.007734,-1.587125,-1.519966
1,1.649293,-1.119261,-1.145840,-1.050163,1.455699,0.893627,2.361086,-0.003657,-0.131770,2.307537,-0.212945,0.990015,-0.101673,1.200555,-0.684813,0.345624,0.875863,-1.029096,-0.867941,-0.980964
2,-0.724760,0.033081,0.131400,0.495870,-2.260084,1.287934,-1.192811,-1.069411,-0.491217,-1.227247,-0.092419,-1.049130,-0.323208,0.197533,-0.009956,0.059702,-0.030089,1.260683,1.229102,0.851853
3,0.109599,-0.757009,-1.128770,-0.527554,0.378104,-1.608467,-1.154402,0.236914,-0.061585,0.825663,1.065985,2.889406,0.786855,-0.656970,-1.008295,-2.101225,-2.163772,-3.275275,0.178095,0.170543
4,0.326490,0.483150,-0.281184,-1.454659,0.222093,-0.577322,1.611002,-0.010222,0.567222,-0.130059,0.108239,-0.091277,-0.670211,1.582457,0.980426,0.783124,-0.908541,-1.102102,-0.697582,0.860034


In [4]:
print type(data)

<class 'pandas.core.frame.DataFrame'>


In [5]:
np_data = np.asarray(data) #converting to numpy array
print np_data.shape 
print type(np_data)

(20000, 20)
<type 'numpy.ndarray'>


In [5]:
#the shape here signifies that there are 20k test samples of 20 features each
#each row signifies 1 sample

In [6]:
labels = pd.read_csv('/home/shashank/Desktop/IIT_D ML/prerequisite_assignment2/label3.csv',header=None)
labels.tail()

,0
19995,1
19996,3
19997,4
19998,1
19999,1


In [7]:
np_labels = np.asarray(labels) #converting to numpy array
print np_labels.shape


(20000, 1)


In [8]:
#this is the output or the target variable....for 19999 samples

In [9]:
print np.unique(np_labels) #this gives us the total number of classes (5) ....thus we get the dimension of one hot encoded vector by this

[0 1 2 3 4]


In [10]:
## code for one hot encoding
temp=0;
labels_onehot = np.zeros((20000,5),dtype=int)

for ix in range(np_labels.shape[0]):
    temp = np_labels[ix]
    labels_onehot[ix,temp]+=1
print labels_onehot[19994:]

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]]


In [11]:
#splitting into train and test data

from sklearn.model_selection import train_test_split


In [12]:
X_train,X_val_test,Y_train,Y_val_test = train_test_split(np_data,labels_onehot,test_size = 0.2)
X_val,X_test,Y_val,Y_test = train_test_split(X_val_test,Y_val_test,test_size=0.5)
Truth = Y_train
Original_input_train = X_train
print 'input(train) shape:', X_train.shape
print 'input(validate)  shape:' , X_val.shape
print 'input(test) shape',X_test.shape
print '----------------------'
print 'output(train) shape:', Y_train.shape
print 'output(validate) shape:',Y_val.shape
print 'output(test) shape:',Y_test.shape

input(train) shape: (16000, 20)
input(validate)  shape: (2000, 20)
input(test) shape (2000, 20)
----------------------
output(train) shape: (16000, 5)
output(validate) shape: (2000, 5)
output(test) shape: (2000, 5)


In [13]:
def Xavier_init(inp_no,next_h_layer_neuron):
    W = np.random.normal(0,1/np.sqrt(inp_no),(inp_no,next_h_layer_neuron))
    B = np.zeros(next_h_layer_neuron,dtype=int)
    return W,B
    

In [14]:
def output_z(W,B,inp):
    Z = np.dot(inp,W) + B
    return Z

In [15]:
def layer_output_act(act_flag,Z):
    #Z = (inp_prev_layer.dot(W))+B #output vector of layer
    if act_flag == 1:
        A = 1/(1 + np.exp(-Z)) #sigmoid     
    elif act_flag == 2:
        A = np.tanh(Z)   #tanh fn
    elif act_flag == 3:
        A = np.maximum(0,Z)  #ReLu fn
    elif act_flag == 4: 
        temp=np.exp(Z)
        A = temp/(np.sum(temp,axis=1,keepdims=True))  #softmax
    else:
        A = Z
    return A

In [16]:
def cost_function(Y,T,batch_size):                     #Y is predicted output of nx5,T is basically labels_onehot
    
    C = -(T*np.log(Y) + (1 - T)*np.log(1-Y)) 
    C_sum = np.sum(C)/batch_size
    return C_sum

In [17]:
#weights/bias init
W1,B1 = Xavier_init(20,20)
W2,B2 = Xavier_init(20,5)

In [18]:
def feedforward(W1,W2,B1,B2,A0):
    
    Z1 = output_z(W1,B1,A0)
    A1 = layer_output_act(2,Z1)
    Z2 = output_z(W2,B2,A1)
    A2 = layer_output_act(4,Z2)
    Y = A2
    
    return Y,A1,Z1

In [19]:
def accuracy(prob_Y,Truth):
    max_pred_index = np.argmax(prob_Y,axis=1)
    max_truth_index = np.argmax(Truth,axis=1)
    correct = np.sum(np.equal(max_pred_index,max_truth_index))
    acc = correct/prob_Y.shape[0]
    return acc

In [22]:
##### TRAINING OF NN #########



batch_size = 512
#no of iterations =16
flag = 0
alpha = 1e-3

for ex in range(600):
    for ix in range(31):
        #data slicing
        A0 = X_train[flag:flag+batch_size, :]
        T = Y_train[flag:flag+batch_size, :]
        
        #forward pass
        Y,A1,Z1 = feedforward(W1,W2,B1,B2,A0)
        
        
        
        
        #Backward pass
        delta_2 = Y-T
        delta_1 = np.dot(delta_2,W2.T) * (1-np.square(np.tanh(Z1)))
        
        
        #calculating dC/dW for different layers
        dC_dW2 = np.dot(A1.T,delta_2)
        dC_dW1 = np.dot(A0.T,delta_1)

        #calculating dC/dB for diff layers
        dC_dB2 = np.sum(delta_2,axis=0)
        dC_dB1 = np.sum(delta_1,axis=0)

        #updation
        W2 = W2 - (alpha*dC_dW2)
        W1 = W1 - (alpha*dC_dW1)
        B2 = B2 - (alpha*dC_dB2)
        B1 = B1 - (alpha*dC_dB1)
        
        flag += 1000
    flag = 0
    
    #cost,feedforward,acc of train
    Y_tot,_,_ = feedforward(W1,W2,B1,B2,Original_input_train)
    C = cost_function(Y_tot,Truth,Y_tot.shape[0])
    ACC = accuracy(Y_tot,Truth)
    
    #feedforward,acc of val ########### VALIDATION DATA ############
    Y_tot_val,_,_ = feedforward(W1,W2,B1,B2,X_val)
    ACC_val = accuracy(Y_tot_val,Y_val)
    
    #feedforward,acc of test
    Y_tot_test,_,_ = feedforward(W1,W2,B1,B2,X_test)
    ACC_test = accuracy(Y_tot_test,Y_test)
    
#     cost_array[ex] = C
#     accu_array[ex] = ACC
    
    print 'Epoch number',ex,':'
    print '---------------------------'
    print 'Train cost is            ', C
    print 'Train accuracy is        ',ACC*100,'%'
    print 'Validation accuracy is   ',ACC_val*100,'%'
    print 'Test accuracy is         ',ACC_test*100,'%'
    print '                  '

Epoch number 0 :
---------------------------
Train cost is             1.93499789206
Train accuracy is         52.775 %
Validation accuracy is    53.9 %
Test accuracy is          52.85 %
                  
Epoch number 1 :
---------------------------
Train cost is             1.88246849952
Train accuracy is         54.4375 %
Validation accuracy is    55.05 %
Test accuracy is          54.55 %
                  
Epoch number 2 :
---------------------------
Train cost is             1.83248917765
Train accuracy is         55.95 %
Validation accuracy is    56.3 %
Test accuracy is          56.9 %
                  
Epoch number 3 :
---------------------------
Train cost is             1.78193933576
Train accuracy is         57.43125 %
Validation accuracy is    57.6 %
Test accuracy is          58.6 %
                  
Epoch number 4 :
---------------------------
Train cost is             1.73198057393
Train accuracy is         59.1375 %
Validation accuracy is    59.15 %
Test accuracy is    

Epoch number 42 :
---------------------------
Train cost is             1.23073901451
Train accuracy is         73.6 %
Validation accuracy is    72.55 %
Test accuracy is          73.6 %
                  
Epoch number 43 :
---------------------------
Train cost is             1.22855171057
Train accuracy is         73.7 %
Validation accuracy is    72.65 %
Test accuracy is          73.2 %
                  
Epoch number 44 :
---------------------------
Train cost is             1.22646615781
Train accuracy is         73.74375 %
Validation accuracy is    72.8 %
Test accuracy is          73.3 %
                  
Epoch number 45 :
---------------------------
Train cost is             1.2244727451
Train accuracy is         73.74375 %
Validation accuracy is    72.75 %
Test accuracy is          73.2 %
                  
Epoch number 46 :
---------------------------
Train cost is             1.22256331861
Train accuracy is         73.70625 %
Validation accuracy is    72.9 %
Test accuracy is  

Epoch number 82 :
---------------------------
Train cost is             1.17963698557
Train accuracy is         74.74375 %
Validation accuracy is    72.7 %
Test accuracy is          74.35 %
                  
Epoch number 83 :
---------------------------
Train cost is             1.17866558125
Train accuracy is         74.7375 %
Validation accuracy is    72.75 %
Test accuracy is          74.2 %
                  
Epoch number 84 :
---------------------------
Train cost is             1.17766748537
Train accuracy is         74.7375 %
Validation accuracy is    72.7 %
Test accuracy is          74.25 %
                  
Epoch number 85 :
---------------------------
Train cost is             1.17663821288
Train accuracy is         74.73125 %
Validation accuracy is    72.65 %
Test accuracy is          74.2 %
                  
Epoch number 86 :
---------------------------
Train cost is             1.17557525259
Train accuracy is         74.73125 %
Validation accuracy is    72.6 %
Test accur

Epoch number 122 :
---------------------------
Train cost is             1.14784859416
Train accuracy is         75.55 %
Validation accuracy is    73.4 %
Test accuracy is          75.1 %
                  
Epoch number 123 :
---------------------------
Train cost is             1.14737311477
Train accuracy is         75.49375 %
Validation accuracy is    73.5 %
Test accuracy is          75.0 %
                  
Epoch number 124 :
---------------------------
Train cost is             1.14690415239
Train accuracy is         75.48125 %
Validation accuracy is    73.65 %
Test accuracy is          74.9 %
                  
Epoch number 125 :
---------------------------
Train cost is             1.14644126848
Train accuracy is         75.48125 %
Validation accuracy is    73.65 %
Test accuracy is          74.85 %
                  
Epoch number 126 :
---------------------------
Train cost is             1.14598400125
Train accuracy is         75.5375 %
Validation accuracy is    73.65 %
Test ac

Epoch number 163 :
---------------------------
Train cost is             1.12320002149
Train accuracy is         76.31875 %
Validation accuracy is    73.45 %
Test accuracy is          75.55 %
                  
Epoch number 164 :
---------------------------
Train cost is             1.12211699448
Train accuracy is         76.35 %
Validation accuracy is    73.55 %
Test accuracy is          75.55 %
                  
Epoch number 165 :
---------------------------
Train cost is             1.12102855715
Train accuracy is         76.39375 %
Validation accuracy is    73.45 %
Test accuracy is          75.35 %
                  
Epoch number 166 :
---------------------------
Train cost is             1.11993945653
Train accuracy is         76.46875 %
Validation accuracy is    73.4 %
Test accuracy is          75.25 %
                  
Epoch number 167 :
---------------------------
Train cost is             1.11885431726
Train accuracy is         76.43125 %
Validation accuracy is    73.45 %
Te

...

Epoch number 407 :
---------------------------
Train cost is             1.07198331841
Train accuracy is         77.6625 %
Validation accuracy is    75.25 %
Test accuracy is          76.55 %
                  
Epoch number 408 :
---------------------------
Train cost is             1.07194588498
Train accuracy is         77.65625 %
Validation accuracy is    75.25 %
Test accuracy is          76.55 %
                  
Epoch number 409 :
---------------------------
Train cost is             1.07190737021
Train accuracy is         77.68125 %
Validation accuracy is    75.25 %
Test accuracy is          76.55 %
                  
Epoch number 410 :
---------------------------
Train cost is             1.07186770346
Train accuracy is         77.6875 %
Validation accuracy is    75.35 %
Test accuracy is          76.55 %
                  
Epoch number 411 :
---------------------------
Train cost is             1.07182681101
Train accuracy is         77.6625 %
Validation accuracy is    75.35 %
T

Epoch number 449 :
---------------------------
Train cost is             1.06869078862
Train accuracy is         77.55625 %
Validation accuracy is    74.95 %
Test accuracy is          76.5 %
                  
Epoch number 450 :
---------------------------
Train cost is             1.06855529615
Train accuracy is         77.5625 %
Validation accuracy is    74.9 %
Test accuracy is          76.45 %
                  
Epoch number 451 :
---------------------------
Train cost is             1.06841687241
Train accuracy is         77.5375 %
Validation accuracy is    74.9 %
Test accuracy is          76.45 %
                  
Epoch number 452 :
---------------------------
Train cost is             1.06827552279
Train accuracy is         77.525 %
Validation accuracy is    74.9 %
Test accuracy is          76.45 %
                  
Epoch number 453 :
---------------------------
Train cost is             1.0681312513
Train accuracy is         77.525 %
Validation accuracy is    74.95 %
Test accu

Epoch number 491 :
---------------------------
Train cost is             1.0596189357
Train accuracy is         77.8125 %
Validation accuracy is    74.95 %
Test accuracy is          76.4 %
                  
Epoch number 492 :
---------------------------
Train cost is             1.05929086644
Train accuracy is         77.80625 %
Validation accuracy is    74.95 %
Test accuracy is          76.4 %
                  
Epoch number 493 :
---------------------------
Train cost is             1.05895945272
Train accuracy is         77.8125 %
Validation accuracy is    74.95 %
Test accuracy is          76.45 %
                  
Epoch number 494 :
---------------------------
Train cost is             1.05862521269
Train accuracy is         77.86875 %
Validation accuracy is    74.9 %
Test accuracy is          76.5 %
                  
Epoch number 495 :
---------------------------
Train cost is             1.05828865653
Train accuracy is         77.8375 %
Validation accuracy is    74.85 %
Test a

Epoch number 533 :
---------------------------
Train cost is             1.04616495018
Train accuracy is         78.225 %
Validation accuracy is    74.9 %
Test accuracy is          77.0 %
                  
Epoch number 534 :
---------------------------
Train cost is             1.0458975174
Train accuracy is         78.19375 %
Validation accuracy is    74.9 %
Test accuracy is          77.0 %
                  
Epoch number 535 :
---------------------------
Train cost is             1.04563418223
Train accuracy is         78.20625 %
Validation accuracy is    74.95 %
Test accuracy is          76.95 %
                  
Epoch number 536 :
---------------------------
Train cost is             1.0453749399
Train accuracy is         78.20625 %
Validation accuracy is    75.0 %
Test accuracy is          76.9 %
                  
Epoch number 537 :
---------------------------
Train cost is             1.0451197715
Train accuracy is         78.19375 %
Validation accuracy is    75.0 %
Test accur

Epoch number 575 :
---------------------------
Train cost is             1.03822013412
Train accuracy is         78.2875 %
Validation accuracy is    75.3 %
Test accuracy is          76.8 %
                  
Epoch number 576 :
---------------------------
Train cost is             1.03810431661
Train accuracy is         78.3 %
Validation accuracy is    75.3 %
Test accuracy is          76.8 %
                  
Epoch number 577 :
---------------------------
Train cost is             1.03799111935
Train accuracy is         78.31875 %
Validation accuracy is    75.3 %
Test accuracy is          76.8 %
                  
Epoch number 578 :
---------------------------
Train cost is             1.03788046816
Train accuracy is         78.3125 %
Validation accuracy is    75.3 %
Test accuracy is          76.8 %
                  
Epoch number 579 :
---------------------------
Train cost is             1.0377722891
Train accuracy is         78.29375 %
Validation accuracy is    75.3 %
Test accuracy 

In [24]:
# ep = np.arange(0,600)

# plt.figure(1)
# plt.plot(ep,cost_array[ep],'r--')
# plt.title('training set cost')
# plt.xlabel('epochs')
# plt.ylabel('cost')

# plt.figure(2)
# plt.plot(ep,accu_array[ep],'b-')
# plt.title('training accuracy')
# plt.xlabel('epochs')
# plt.ylabel('accuracy')